<a href="https://colab.research.google.com/github/Muhammad-Usama-07/agentic_ai/blob/main/Different_Basic_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packages

In [ ]:
!pip install -U langchain langchain-openai python-dotenv langchain-groq
!pip install -U langchain langchain-core langchain-community langchain-groq
!pip install -U langchain langchain-core langchain-community langchain-groq langchain-classic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: langgraph-prebuilt
    Found existing installation: langgraph-prebuilt 1.0.6
    Uninstalling langgraph-prebuilt-1.0.6:
      Successfully uninstalled langgraph-prebuilt-1.0.6
  Attempting uninstall: langgraph
    Found existing installation: langgraph 1.0.6
    Uninstalling langgraph-1.0.6:
      Successfully uninstalled langgraph-1.0.6
  Attempting uninstall: langchain
    Found existing installation: langchain 1.2.4
    Uninstalling langchain-1.2.4:
      Successfully uninstalled langchain-1.2.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.0 MB/s eta 0:00:00

# Agents Description

## Agent 1
- Take question from the user
- Give answer by using a tool(a method)

# Agent 1 (Langchain + OpenAI)

#### set key

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""


In [ ]:
print('aaa')

aaa


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv

load_dotenv()

# Define the tool
@tool
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

# LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# Create agent
agent = create_openai_functions_agent(
    llm=llm,
    tools=[get_weather],
    prompt=prompt
)

# Agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[get_weather],
    verbose=True
)

# Run
result = agent_executor.invoke({
    "input": "What is the weather in San Francisco?"
})

print(result["output"])


# Agent 1 (Langchain + Groq)

In [ ]:
import os

os.environ["GROQ_API_KEY"] = ""


In [ ]:
from langchain_groq import ChatGroq
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import ToolMessage

#
# ------------------------- TOOL -------------------------
@tool
def get_code() -> str:
    """Returns a secret code unknown to the model."""
    print("TOOL EXECUTED: get_code")
    return "cod-12345"

tools = [get_code]

# ------------------------- LLM -------------------------
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)

llm_with_tools = llm.bind_tools(tools)

# ------------------------- PROMPT -------------------------
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You do NOT know the secret code. "
        "The ONLY way to get it is by calling the provided tool. "
        "You MUST use the tool to answer."
    ),
    ("human", "{input}")
])

# ------------------------- USER QUESTION -------------------------
messages = prompt.invoke(
    {"input": "What is the secret code?"}
).to_messages()

# ------------------------- STEP 1 (REQUEST TOOL) -------------------------
response = llm_with_tools.invoke(messages)

print("\nModel response:")
print(response)

# ------------------------- TOOL EXECUTION -------------------------
if response.tool_calls:
    tool_call = response.tool_calls[0]

    print("\nTool requested by model:")
    print(tool_call)

    # Execute tool
    tool_result = get_code.invoke(tool_call["args"])

    # ------------------------- STEP 2 (FINAL ANSWER) -------------------------
    final_response = llm.invoke([
        *messages,
        response,
        ToolMessage(
            content=tool_result,
            tool_call_id=tool_call["id"]
        )
    ])

    print("\nFINAL ANSWER:")
    print(final_response.content)
else:
    print("Model did NOT call the tool")



Model response:
content='' additional_kwargs={'tool_calls': [{'id': 'ymz3y8sag', 'function': {'arguments': '{}', 'name': 'get_code'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 242, 'total_tokens': 250, 'completion_time': 0.015676753, 'completion_tokens_details': None, 'prompt_time': 0.020151408, 'prompt_tokens_details': None, 'queue_time': 0.038662682, 'total_time': 0.035828161}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_bebe2dd4fb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019bf4fb-ea3d-75f2-b8f9-a285683d45d7-0' tool_calls=[{'name': 'get_code', 'args': {}, 'id': 'ymz3y8sag', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 242, 'output_tokens': 8, 'total_tokens': 250}

Tool requested by model:
{'name': 'get_code', 'args': {}, 'id': 'ymz3y8sag', 'type': 'tool_call'}
TOOL EXECUTED: get_code

FINAL ANSWER:


# Agent 2 (Langchain + groq)

1. Chatting with LLM
2. calling tool to provide specific string


In [ ]:
from langchain_groq import ChatGroq
from langchain.tools import tool
from langchain_core.messages import ToolMessage, HumanMessage, AIMessage

# ------------------------- TOOL -------------------------
@tool
def get_code() -> str:
    """Returns a secret code. Use ONLY when the user asks for the code or secret."""
    print("TOOL EXECUTED: get_code")
    return "cod-12345"

tools = [get_code]

# ------------------------- LLM -------------------------
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)

llm_with_tools = llm.bind_tools(tools)

# ------------------------- SYSTEM PROMPT -------------------------
system_prompt = (
    "You are a general-purpose assistant.\n"
    "- You can answer normal questions directly.\n"
    "- You do NOT know any secret or code.\n"
    "- If (and ONLY if) the user asks for a secret code, secret, or code, "
    "you MUST call the provided tool to get it.\n"
    "- Do not call the tool for greetings or general conversation."
)

# ------------------------- CHAT LOOP -------------------------
print("\nAgent started\n")

chat_history = []

while True:
    print("\nChoose an option:")
    print("1. Continue chatting")
    print("2.  Exit")

    choice = input("Enter 1 or 2: ").strip()

    if choice == "2":
        print("\nExiting chat. Goodbye!")
        break

    elif choice == "1":
        user_input = input("\nYou: ").strip()

        messages = [
            {"role": "system", "content": system_prompt},
            *chat_history,
            HumanMessage(content=user_input)
        ]

        # -------- Step 1: Model decides
        response = llm_with_tools.invoke(messages)

        # -------- Step 2: Tool execution (ONLY if requested)
        if response.tool_calls:
            tool_call = response.tool_calls[0]

            tool_result = get_code.invoke(tool_call["args"])

            final_response = llm.invoke([
                *messages,
                response,
                ToolMessage(
                    content=tool_result,
                    tool_call_id=tool_call["id"]
                )
            ])
        else:
            final_response = response

        print("\nAssistant:", final_response.content)

        chat_history.append(HumanMessage(content=user_input))
        chat_history.append(AIMessage(content=final_response.content))

    else:
        print("\nInvalid choice. Please enter 1 or 2.")



Agent started


Choose an option:
1. Continue chatting
2.  Exit
Enter 1 or 2: 1

You: Hi

Assistant: Hello. How can I assist you today?

Choose an option:
1. Continue chatting
2.  Exit
Enter 1 or 2: 2

Exiting chat. Goodbye!


### 3. close program when user say good bye or something?

In [14]:
# from langchain_groq import ChatGroq
# from langchain.tools import tool
# from langchain_core.messages import ToolMessage, HumanMessage, AIMessage

# # ------------------------- TOOL -------------------------
# @tool
# def get_code() -> str:
#     """Returns a secret code. Use ONLY when the user asks for a secret or code."""
#     print("TOOL EXECUTED: get_code")
#     return "cod-12345"

# tools = [get_code]

# # ------------------------- LLM -------------------------
# llm = ChatGroq(
#     model="llama-3.3-70b-versatile",
#     temperature=0
# )

# llm_with_tools = llm.bind_tools(tools)

# # ------------------------- SYSTEM PROMPT -------------------------
# system_prompt = (
#     "You are a general-purpose assistant.\n"
#     "- You can answer normal questions directly.\n"
#     "- You do NOT know any secret or code.\n"
#     "- ONLY if the user asks for a secret or code, you MUST call the tool.\n"
#     "- Do NOT call the tool for greetings or general conversation."
# )

# # ------------------------- EXIT WORDS -------------------------
# exit_words = {"bye", "goodbye", "exit", "quit", "stop"}

# # ------------------------- CHAT LOOP -------------------------
# print("Agent started. Type 'bye' to exit.\n")

# chat_history = []

# while True:
#     user_input = input("You: ").strip()

#     if user_input.lower() in exit_words:
#         print("\nAssistant: Goodbye! Have a great day.")
#         break

#     messages = [
#         {"role": "system", "content": system_prompt},
#         *chat_history,
#         HumanMessage(content=user_input)
#     ]

#     # Step 1: Model decides
#     response = llm_with_tools.invoke(messages)

#     # Step 2: Execute tool only if requested
#     if response.tool_calls:
#         tool_call = response.tool_calls[0]
#         tool_result = get_code.invoke(tool_call["args"])

#         final_response = llm.invoke([
#             *messages,
#             response,
#             ToolMessage(
#                 content=tool_result,
#                 tool_call_id=tool_call["id"]
#             )
#         ])
#     else:
#         final_response = response

#     print("\nAssistant:", final_response.content, "\n")

#     chat_history.append(HumanMessage(content=user_input))
#     chat_history.append(AIMessage(content=final_response.content))
from langchain_groq import ChatGroq
from langchain.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# -------------------- TOOL --------------------
@tool
def get_code() -> str:
    """Returns a secret code. Use ONLY when the user asks for a code or secret."""
    print("TOOL EXECUTED: get_code")
    return "cod-12345"

tools = [get_code]

# -------------------- MODELS --------------------
chat_llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)

judge_llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)

chat_llm_with_tools = chat_llm.bind_tools(tools)

# -------------------- EXIT DETECTION FUNCTION --------------------
def should_exit(user_input: str) -> bool:
    """
    Uses the LLM to decide if the user wants to end the conversation.
    Returns True if the intent is to exit, otherwise False.
    """

    prompt = f"""
    You are an intent classifier.
    Decide if the following message means the user wants to END the conversation.

    User message:
    "{user_input}"

    Reply with ONLY one word:
    YES or NO
    """

    response = judge_llm.invoke(prompt).content.strip().upper()
    return response == "YES"

# -------------------- SYSTEM PROMPT --------------------
system_prompt = (
    "You are a helpful general-purpose assistant.\n"
    "You can answer any normal question.\n"
    "You do NOT know any secret or code.\n"
    "ONLY if the user asks for a secret or code, you MUST call the tool.\n"
)

# -------------------- CHAT LOOP --------------------
print("Agent started. Chat freely — say anything when you want to leave.\n")

chat_history = []

while True:
    user_input = input("You: ").strip()

    # LLM decides whether to exit
    if should_exit(user_input):
        print("\nAssistant: Goodbye! Take care.")
        break

    messages = [
        {"role": "system", "content": system_prompt},
        *chat_history,
        HumanMessage(content=user_input)
    ]

    response = chat_llm_with_tools.invoke(messages)

    if response.tool_calls:
        tool_call = response.tool_calls[0]
        tool_result = get_code.invoke(tool_call["args"])

        final_response = chat_llm.invoke([
            *messages,
            response,
            ToolMessage(
                content=tool_result,
                tool_call_id=tool_call["id"]
            )
        ])
    else:
        final_response = response

    print("\nAssistant:", final_response.content, "\n")

    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=final_response.content))


Agent started. Chat freely — say anything when you want to leave.

You: Hello 

Assistant: Hello. How can I assist you today? 

You: who are yoo

Assistant: I'm a helpful general-purpose assistant. I can provide information, answer questions, and help with a wide range of topics. I'm here to assist you with anything you need, so feel free to ask me anything. How can I help you today? 

You: what are you doing

Assistant: I'm currently chatting with you and waiting for your questions or topics you'd like to discuss. I'm ready to provide information, answer questions, and help with anything you need. What's on your mind? 

You: ok, thanks bye

Assistant: Goodbye! Take care.
